# Dashboard to explore data with ipywidget controls


### Set up requirements, import packages

In [1]:
%load_ext autoreload
%autoreload

import os
if not os.path.basename(os.getcwd()) == "datenguide-python":
    os.chdir("..")
    
    
from datenguidepy.query_helper import get_regions, get_statistics
from datenguidepy import Query
import ipywidgets as widgets
from ipywidgets import interact, interactive
%matplotlib inline

pd.set_option('display.max_colwidth', 150)

### Create dashboard

Using ipywidgets you can browse data types here. Opening the list of regions will take a few seconds.

If the result remains empty the selected region have no corresponding data for the selected statistic.

First lets select the NUTS region level. Based on this value the region selector will list the available regions.


#### Choosing region and statistic

After selecting NUTS level choose from the list of regions and statistics.

In [2]:
all_statistics = get_statistics().reset_index().loc[:,["short_description", "statistic"]]
all_statistics = sorted([tuple(x) for x in all_statistics.to_numpy()])

In [3]:
def nuts_selector(nuts_level):
    
    regions = get_regions().query(f"level == {nuts_level}").reset_index()[["name", "region_id"]]
    regions = sorted([tuple(x) for x in regions.to_numpy()])

    @interact
    def get_statistics_dashboard(region=regions, statistic=all_statistics):

        q = Query.region(region)

        try:
            field = q.add_field(statistic)
        except KeyError as e:
            return(f"Statistic not available: {e}")
        
        return q.results()

interact(nuts_selector, nuts_level=[('NUTS 1', "'nuts1'"), ('NUTS 2', "'nuts2'"), ('NUTS 3', "'nuts3'")]);

interactive(children=(Dropdown(description='nuts_level', options=(('NUTS 1', "'nuts1'"), ('NUTS 2', "'nuts2'")…